<a href="https://colab.research.google.com/github/vishesh711/Development-of-a-Long-Context-Transformer-within-RAG-for-Enhanced-Document-Understanding/blob/main/RAG_TESTING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Setup



In [2]:
!pip install -q sentence-transformers
!pip install -q wikipedia-api
!pip install -q numpy
!pip install -q scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### Load the Embedding Model:

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

### Fetch Text Content from Wikipedia:



In [4]:
from wikipediaapi import Wikipedia
wiki = Wikipedia('RAGBot/0.0', 'en')
doc = wiki.page('Hayao_Miyazaki').text
paragraphs = doc.split('\n\n') # chunking


In [5]:
import textwrap


In [6]:
for i, p in enumerate(paragraphs):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")


-----------------------------------------------------------------
Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, [mijaꜜzaki hajao]; born January 5, 1941) is a Japanese
animator, filmmaker, and manga artist. A founder of Studio Ghibli, he has attained international
acclaim as a masterful storyteller and creator of Japanese animated feature films, and is widely
regarded as one of the most accomplished filmmakers in the history of animation. Born in Tokyo City,
Miyazaki expressed interest in manga and animation from an early age. He joined Toei Animation in
1963, working as an inbetween artist and key animator on films like Gulliver's Travels Beyond the
Moon (1965), Puss in Boots (1969), and Animal Treasure Island (1971), before moving to A-Pro in
1971, where he co-directed Lupin the Third Part I (1971–1972) alongside Isao Takahata. After moving
to Zuiyō Eizō (later Nippon Animation) in 1973, Miyazaki worked as an animator on World Masterpiece
Theater and directed the television series Fut

### Embed the Document:

In [7]:
docs_embed = model.encode(paragraphs, normalize_embeddings=True)

In [8]:
docs_embed.shape

(20, 768)

In [9]:
docs_embed[0]

array([ 9.97676235e-03, -4.28699143e-03,  4.71468568e-02, -1.87339671e-02,
        1.97542529e-03, -2.24169772e-02,  4.35171984e-02,  4.56002243e-02,
        5.11342771e-02, -9.35613923e-03, -3.20787588e-03,  7.57397478e-03,
        3.12514007e-02,  1.78152509e-02, -2.10417528e-02,  1.44400522e-02,
        1.30389398e-02, -3.35419886e-02, -2.84356605e-02, -3.02529801e-02,
        4.69961250e-03,  9.93409473e-03, -2.59442125e-02,  3.31464224e-02,
       -7.64328381e-03, -4.05385979e-02, -3.72275487e-02, -1.73108410e-02,
       -2.98674293e-02, -1.60576054e-03,  2.54417993e-02, -9.97081492e-03,
        1.07298065e-02,  1.19927386e-02, -2.67969109e-02, -5.62506393e-02,
       -3.94381732e-02,  5.29515371e-02, -2.24437192e-02, -3.57856750e-02,
       -7.25627914e-02,  1.01334974e-02, -2.98694000e-02, -1.57779381e-02,
       -5.15344441e-02,  1.84595194e-02,  1.81350745e-02, -3.73585261e-02,
        9.20160499e-04, -1.55232521e-02, -4.18025218e-02,  2.15176004e-03,
        1.80185377e-03, -

### Embed the Query:

In [10]:
query = "What was Studio Ghibli's first film?"
query_embed = model.encode(query, normalize_embeddings=True)


In [11]:
query_embed.shape

(768,)

### Find the Closest Paragraphs to the Query:



In [12]:
import numpy as np
similarities = np.dot(docs_embed, query_embed.T)

In [13]:
similarities.shape

(20,)

In [14]:
similarities

array([0.5495362 , 0.45591563, 0.5105508 , 0.5068357 , 0.4823144 ,
       0.63270956, 0.55179083, 0.63175017, 0.5288352 , 0.539761  ,
       0.4757053 , 0.46614614, 0.4197607 , 0.432001  , 0.39472258,
       0.46836787, 0.4371642 , 0.43962178, 0.19419616, 0.4650128 ],
      dtype=float32)

In [15]:
top_3_idx = np.argsort(similarities, axis=0)[-3:][::-1].tolist()


In [16]:
top_3_idx

[5, 7, 6]

In [17]:
most_similar_documents = [paragraphs[idx] for idx in top_3_idx]

In [18]:
CONTEXT = ""
for i, p in enumerate(most_similar_documents):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")
  CONTEXT += wrapped_text + "\n\n"

-----------------------------------------------------------------
Studio Ghibli Early films (1985–1995) Following the success of Nausicaä of the Valley of the Wind,
Miyazaki and Takahata founded the animation production company Studio Ghibli on June 15, 1985, as a
subsidiary of Tokuma Shoten, with offices in Kichijōji designed by Miyazaki. Miyazaki named the
studio after the Caproni Ca.309 and the Italian word meaning "a hot wind that blows in the desert";
the name had been registered a year earlier. Suzuki worked for Studio Ghibli as producer, joining
full-time in 1989, while Topcraft's Tōru Hara became production manager; Suzuki's role in the
creation of the studio and its films has led him to being occasionally named a co-founder, and Hara
is often viewed as influential to the company's success. Yasuyoshi Tokuma, the founder of Tokuma
Shoten, was also closely related to the company's creation, having provided financial backing.
Topcraft had been considered as a partner to produce Mi

In [19]:
query = "What was Studio Ghibli's first film?"

In [20]:
prompt = f"""
use the following CONTEXT to answer the QUESTION at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

CONTEXT: {CONTEXT}
QUESTION: {query}

"""

In [21]:
!pip install -q openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00


In [22]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.5 MB/s eta 0:00:00


In [23]:
!export GROQ_API_KEY=<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>


/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `export GROQ_API_KEY=<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>'


In [24]:
import os
os.environ["GROQ_API_KEY"] = "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"


In [25]:
from groq import Groq

client = Groq()


In [26]:
from groq import Groq

# Initialize the client with your API key directly
client = Groq(api_key="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")

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What was Studio Ghibli's first film?",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)


Studio Ghibli's first film was "Nausicaa of the Valley of the Wind" (Kaze no Tani no Nausicaa), released in 1984. It was directed by Hayao Miyazaki, co-founder of the studio, and it was based on Miyazaki's own manga series of the same name.


In [27]:
import numpy as np

# Assuming `docs_embed` is your matrix of paragraph embeddings, and `query_embed` is the embedding of your query.

# Step 1: Calculate Cosine Similarity
# Normalize embeddings for cosine similarity
docs_norm = np.linalg.norm(docs_embed, axis=1, keepdims=True)
query_norm = np.linalg.norm(query_embed)
normalized_docs = docs_embed / docs_norm
normalized_query = query_embed / query_norm

# Compute cosine similarities
similarities = np.dot(normalized_docs, normalized_query.T)

# Step 2: Get Top-N Similar Documents
top_n = 3  # Set the number of top documents to retrieve
top_n_idx = np.argsort(similarities, axis=0)[-top_n:][::-1]  # Indices of top-N similar documents

# Step 3: Display Results
print("Top similar paragraphs based on cosine similarity:\n")
for i, idx in enumerate(top_n_idx):
    similarity_score = similarities[idx]
    paragraph = paragraphs[idx]
    print(f"Rank {i+1} - Similarity Score: {similarity_score:.4f}")
    print("-----------------------------------------------------------------")
    print(paragraph)
    print("-----------------------------------------------------------------\n")


Top similar paragraphs based on cosine similarity:

Rank 1 - Similarity Score: 0.6327
-----------------------------------------------------------------
Studio Ghibli
Early films (1985–1995)
Following the success of Nausicaä of the Valley of the Wind, Miyazaki and Takahata founded the animation production company Studio Ghibli on June 15, 1985, as a subsidiary of Tokuma Shoten, with offices in Kichijōji designed by Miyazaki. Miyazaki named the studio after the Caproni Ca.309 and the Italian word meaning "a hot wind that blows in the desert"; the name had been registered a year earlier. Suzuki worked for Studio Ghibli as producer, joining full-time in 1989, while Topcraft's Tōru Hara became production manager; Suzuki's role in the creation of the studio and its films has led him to being occasionally named a co-founder, and Hara is often viewed as influential to the company's success. Yasuyoshi Tokuma, the founder of Tokuma Shoten, was also closely related to the company's creation, havi